In [1]:
import requests
import json

url = '10.168.20.20:5010'
entries = ['5b689a74ea0637002f6a4e31']
rolesa = '5b688de7ea0637002f6a4e30'

url = 'localhost:5010'
entries = ['5b3ec47706f9e50f2f36acc8', '5b3fd54906f9e50f2f36acc9']
rolesa = '593f49be42d3ed97390f25a7'

In [2]:
pipeline = [
    {'$match': {'_id': {'$in': entries}, 'roles._id': rolesa}},
    {
        '$graphLookup': {
            'from': 'applications',
            'startWith': '$deps._id',
            'connectFromField': 'deps._id',
            'connectToField': '_id',
            'as': 'nodes',
            'maxDepth': 10,
            'depthField': 'steps'
        }
    },
    {'$project': {
                    'name': 1, 'family': 1, 
                    'servers': 1,
                    'deps._id': 1, 'deps.endpoint': 1, 
                    'nodes._id': 1, 'nodes.deps': 1, 
                    'nodes.steps': 1, 'nodes.name': 1, 
                    'nodes.family': 1, 'nodes.servers': 1,
                    'nodes.provider': 1, 'nodes.datacenters': 1
                 }
    }
];

jpipeline = json.dumps(pipeline)

data = requests.post('http://%s/aggregate' % url, json={'entity': 'applications', 'pipeline': jpipeline})
data = data.json()

In [3]:
import sys
import os
import numpy as np
cwd = os.getcwd()
spltted = cwd.split('/')[:-1]
sys.path.append('/'.join(spltted))

import networkx as nx
from app.services.gridOrchestrator import GridOrchestrator
from app.libs.network.baseNetwork import BaseNetwork
from app.libs.histograms.histogram import Histogram
from app.libs.histograms.grid import GridHistogram

from app.libs.transformDict import append_servers, transform_dict

from svgwrite import Drawing

network = BaseNetwork()
network.make(data.get('items')).get_graph()

In [4]:
mapping_style = {
    'aws': '.aws{color: #f58536;fill: #c4692c;}',
    'premise': '.premise{color: #5b5b5b;fill: #8e8e8e;}',
    'openstack': '.openstack{color: #ce081f;fill: #af071a;}',
    'azure': '.azure{color: #1566bf;fill: #7396bc;}',
    'default': '.dark{fill: currentColor;}.darker{filter: url(#brightness)}.2darker{filter: url(#darker)}text{font-family:Arial, Helvetica, sans-serif; fill: #672064;}.white{fill:#fff;}.tfont{fill: #e8e8e8}.tb-back{stroke-width: 1; stroke: #a0a0a0; fill: #000; fill-opacity: 0.5}'
}


class LoadTemplates(object):
    
    def __init__(self, path):
        self._path = path
        self._mapp = {}
        
        self.crawler()
    
    def __call__(self):
        return self._mapp
        
    def crawler(self):
        for (dirpath, dirnames, filenames) in os.walk(self._path):

            if filenames:
                spl = dirpath.split('/')[-1]
                self.iter_files(spl, filenames)

    
    def without_prefix(self, label):
        spt = label.split('.')[:-1]
        return ''.join(spt)
    
    def get_prefix(self, pfx):
        if pfx:
            return '%s.' % pfx
        
        return ''
    def iter_files(self, prefix, filenames):
        
        for file in filenames:
            
            name = '%s%s/%s' % (self._path, prefix, file)
            key= '%s%s' % (self.get_prefix(prefix), self.without_prefix(file))
            with open(name) as f:
                data = json.load(f)    
                self._mapp[key] = data
        
        

mypath = '/'.join(spltted) + '/assets/symbol/'

mapping = LoadTemplates(mypath)()

In [5]:
class Creator(object):
    def __init__(self, assets):
        self._assets = assets

    def make(self):
        points = self.get_args('points')
              
        if points:
            points = self.getPoints(points)
            self.set_args('points', points)
            
        return self._assets
    
    def get_args(self, label):
        return self._assets.get(label)
    
    def set_args(self, label, value):
        self._assets[label] = value
    
    def getPoints(self, points):
        prepared = []
        splitted = points.split(' ')

        tmp = None
        for item in splitted:
            if tmp:
                cons = (tmp, item)
                prepared.append(cons)
                tmp = None
            else:
                tmp = item
        
        return prepared

class ManageAssets(object):
    def __init__(self, symbol, creator=Creator):
        self._symbol_assets = symbol
        self._creator = creator
    
    def polyline(self, opt={}):
        return self._symbol_assets.dwg.polygon(**opt)
    
    def path(self, opts):
        return self._symbol_assets.dwg.path(**opts)
    
    def create(self, configs):
        objs = []
        items = configs.get('els')
        
        for item in items:
            args = item.get('args')
            shape = item.get('shape')
            opts = self._creator(args).make()
            
            obj = getattr(self, shape)(opts)
            objs.append(obj)
        
        return objs

In [6]:
import math

class CalCosine(object):
    
    cos = math.cos(math.radians(30.4))
    
    @staticmethod
    def cal(cat):
        hip =  cat / CalCosine.cos
        return math.sqrt((hip**2) - (cat**2))


class CalSine(object):
    
    cos = math.sin(math.radians(30.4))
    
    @staticmethod
    def cal(cat):
        hip =  cat / CalSine.cos
        return math.sqrt((hip**2) - (cat**2))
    
class MatrixRotation(object):
    
    def __init__(self, max_x, size, off, grid={}):
        self._size = size
        self._off = off
        self._mx = (size[0] * off[0]) / 2
        self._cosi = CalCosine.cal(self._mx)
        self._grid = grid
        
        self._rect_y = ((max_x-1) * self._cosi)
        
    def cal_offy(self, x, y):
        return self._rect_y + (y * self._cosi) - (x * self._cosi)
    
    def cal_offx(self, x, y):
        return (y * self._mx) + (x * self._mx)

In [7]:
import copy
import math

class DrawArea(object):
    def __init__(self, off, size, max_x, max_y, grid, mrotation=MatrixRotation):
        self._off = off
        self._size = size
        self._max_x = max_x
        self._max_y = max_y
        self._grid = grid
        
        self._mrotation = MatrixRotation(self._max_x, size, off)

    def area_x(self):
        menus = self.shw_empty()
        
        cal = self._mrotation.cal_offx(self._max_x, self._max_y) + self._size[0] 
        return cal - (menus * self._size[0])
    
    def area_y(self):
        return self._mrotation.cal_offy(0, self._max_y) + self._size[1]
    
    def area(self):
        area_x = self.area_x()
        area_y = self.area_y()
        return (area_x, area_y)

    def shw_empty(self):
        x = self._max_x
        y = self._max_y
        
        counter = 0

        while counter < self._max_x:
            lst = self.list_checked(counter, x, y)
            for pos in lst:
                checked = self.check_empty(*pos)
                if not checked:
                    return counter
                
            counter += 1
        return counter-1
    
    def list_checked(self, counter, x, y):
        '''
        counter 0 -> 1 item
        counter 1 -> 2 item
        '''
        pos = set()
        if counter == 0:
            pos.add((x, y))
        
        if counter >= 1:
            pos.add((x, y-counter))
            pos.add((x-counter, y))
        
        if counter > 1:
            ct = counter - 1 #remove 
            for item in range(ct, 0, -1):
                p1 = (x, y-counter+item)
                p2 = (x-counter+item, y)
                pos.add(p1)
                pos.add(p2)
        
        return pos
        
        
    def check_empty(self, x, y):
        obj = x in self._grid and y in self._grid[x]
        return (not obj) and (obj != '-')
        

class ManageStyle(object):
    
    def __init__(self, draw, smap=mapping_style):
        self.dwg = draw
        self._used_styles = []
        self._smap = smap
    
    def stylish(self, template):
        
        if template not in self._used_styles:
            tpl = self.get_template(template)
            self.attach_style(tpl)
            
            self._used_styles.append(template)
    
    def get_template(self, name, dft='default'):
        if name in self._smap:
            return self._smap[name]
        
        return self._smap[dft] 
            
    def attach_style(self, styles):
        stl = self.dwg.style(styles)
        self.dwg.defs.add(stl)
    
class Symbol(object):
    
    def __init__(self, draw=Drawing, mstyle=ManageStyle):
        self.dwg = draw
        self._s_basket = {}
        self._proportion = {}
        
        self._factor_proportion = 10
        
        self._manager_style = mstyle(self.dwg)
    
    def create_symbol(self, name, objs, viewbox):
        symbol = self.dwg.symbol(id=name)
        self.dwg.defs.add(symbol)
        
        symbol.viewbox(*viewbox)
        symbol.fit(vert='bottom')
        
        for obj in objs:
            symbol.add(obj)
        
        self._s_basket[name] = symbol
    
    def create_marker(self, name, objs, viewbox, opts):
        symbol = self.dwg.marker(id=name, **opts)
        self.dwg.defs.add(symbol)
        
        symbol.viewbox(*viewbox)
        
        for obj in objs:
            symbol.add(obj)
        
        self._s_basket[name] = symbol
    
    def create_group(self, name):
        g = self.dwg.defs.add(self.dwg.g(id=name))
        return g

        
    def get_symbol(self, name):
        return self._s_basket[name]
        
    def use_symbol(self, name, pos, opts={}):
        syb = self.get_symbol(name)
        return self.dwg.use(syb, insert=pos, **opts)
    
    def get_proportion(self, k):
        return self._proportion[k]
    
    def set_proportion(self, k, val):
        x = val[0] / self._factor_proportion
        y = val[1] / self._factor_proportion
        self._proportion[k] = (x, y)
        
    def square(self, pos, size, opts={}):
        return self.dwg.rect(insert=pos, size=size, **opts)
    
    def polyline(self, points, opts={}):
        return self.dwg.polyline(points=points, **opts)
    
    def text(self, title, pos, opt={}):
        return self.dwg.text(title, insert=pos, **opt)
    
    def multiline(self, ltxt, pos, opts={}):
        if not isinstance(ltxt, list):
            raise Exception("Need to be a list")
            
        atext = self.text("", pos)
        
        for opts in ltxt:
            label = opts.get('text')
            del opts['text']
            
            atext.add(self.tspan(label, opts))
        
        return atext
        
    def tspan(self, text, opts={}):
        return self.dwg.tspan(text, **opts)

    def line(self, pos, opts={}):
        cx, cy = pos
        return self.dwg.line(start=cx, end=cy, **opts)

    def path(self, d, opts):
        return self.dwg.path(d, **opts)
    
    def conn(self, d):
        opts = {
            'stroke_width': "1",
            'stroke': "black",
            'fill': "none",
            'opacity': 0.8,
            "marker_end": self.get_symbol('markers.arrow').get_funciri()
        }
        
        return self.path(d, opts)
    
    def brightness(self, idd="brightness", force=0.6):
        flt=self.dwg.filter(id=idd)
        dflt = self.dwg.defs.add(flt)
        
        feComp = dflt.feComponentTransfer()
        feComp.feFuncR('linear', slope=force)
        feComp.feFuncG('linear', slope=force)
        feComp.feFuncB('linear', slope=force)
        return flt
        
        
class SymbolAssets(Symbol):
    
    def __init__(self, draw, assets=mapping, manager=ManageAssets):
        super().__init__(draw)
        
        self._map_assets = assets
        self._manager = manager(self)
    
    def find_assets(self, key, dft='default'):
        if key not in self._map_assets:
            key = dft
        
        return key
    
    def asset_marker(self, asset, opts={}):
        asset = self.find_assets(asset)
        
        if not asset in self._s_basket:
            configs = copy.deepcopy(self._map_assets[asset])
            create = self._manager.create(configs)

            viewbox = configs.get('viewBox')
            self.create_marker(asset, create, viewbox, opts)
    
    def asset(self, asset, template, pos, size, opts={}):
        asset = self.find_assets(asset)
        
        if not asset in self._s_basket:
            configs = copy.deepcopy(self._map_assets[asset])
            create = self._manager.create(configs)

            viewbox = configs.get('viewBox')
            self.create_symbol(asset, create, viewbox)
            
            self.set_proportion(asset, (viewbox[2], viewbox[3]))
            
            self._manager_style.stylish(template)
        
        
        factor = self.get_proportion(asset)
        opts['size'] = size
        
        if template:
            opts['class_'] = template

        return self.use_symbol(asset, pos, opts)

In [8]:
#10 = 100 / x

#50 = 100 / 2
#50 = 100 / x -> x = 100 / 50
#50 = x / 2 -> x = 50 * 2

In [9]:
from operator import itemgetter
from random import randint
from decimal import *

class CalGrid(object):
    def __init__(self, size, gsize=2):
        
        self._map = None
        
        self._gsize = gsize
        self._size = size
        
        self._nsize = self.node_size()
        self._base = self.cal_base()
        self._cato = self.cal_cosins()
        
    def node_size(self):
        prop = self._gsize
        
        if prop == 1:
            prop += 1
        return (self._size[0] / prop, self._size[1] / prop)
        
    def get_size(self):
        return self._gsize ** 2
    
    def get_gsize(self):
        return self._gsize
    
    def get_nsize(self):
        return self._nsize
        
    def cal_base(self):
        return [self._nsize[x]/2 for x in range(2)]
    
    def cal_cosins(self):
        cata = self._size[0] / self._gsize
        cos1 = CalCosine.cal(cata)
        cos2 = CalCosine.cal(self._base[0])
        return cos1, cos2
    
    def get_position(self, pos):
        mapper = self.build_map()
        return [mapper[x][pos] for x in range(2)]  

class CalGridOne(CalGrid):
    def __init__(self, size, gsize=2):
        super().__init__(size, gsize)
    
    def build_map(self):
        
        if self._map == None:
            xbase, ybase = [self._base[x] for x in range(2)]
            cato_box, cato_node = self._cato
            
            yspace = (self._size[1] / self._gsize)
            
            grid_x = [xbase]
            grid_y = [yspace - (cato_box / 2)]

            self._map = grid_x, grid_y
            
        return self._map
    
class CalGridFourth(CalGrid):
    def __init__(self, size, gsize=2):
        super().__init__(size, gsize)
     
    def build_map(self):
        
        if self._map == None:
            xbase, ybase = [self._base[x] for x in range(2)]
            cato_box, cato_node = self._cato
            
            yspace = (self._size[1] / self._gsize)
            
            grid_x = (xbase, xbase*2, 0, xbase)
            grid_y = (
                yspace - cato_box, 
                yspace - cato_box + cato_node, 
                yspace - cato_box + cato_node,  
                yspace
            )

            self._map = grid_x, grid_y
            
        return self._map

class CalGridNineth(CalGrid):
    def __init__(self, size, gsize=3):
        super().__init__(size, gsize)
     
    def build_map(self):
        
        if self._map == None:
            xbase, ybase = [self._base[x] for x in range(2)]
            cato_box, cato_node = self._cato
            
            yspace = self._size[1] - self._nsize[1]
            
            grid_x = (
                xbase*2, xbase*3, xbase*4, 
                xbase, xbase*2, xbase*3, 
                0, xbase, xbase*2
            )
            
            grid_y = (
                yspace - cato_box - cato_box, 
                yspace - cato_box + cato_node - cato_box,  
                yspace - cato_box,
                
                yspace - cato_box + cato_node - cato_box, 
                yspace - cato_box,
                yspace - cato_box + cato_node,
                
                yspace - cato_box,  
                yspace - cato_box + cato_node,
                yspace
            )

            self._map = grid_x, grid_y
            
        return self._map

    
class FactoryCalGrid(object):
    
    @staticmethod
    def caller(qtd, size):
        gsize = FactoryCalGrid.grid_size(qtd)
        grid = (CalGridOne, CalGridOne, CalGridFourth, CalGridNineth)
        
        if gsize < len(grid):
            call = grid[gsize]
        else:
            call = grid[-1]
            
        return call(size)
    
    @staticmethod
    def grid_size(qtd):
        return math.ceil(math.sqrt(qtd))
        
    
class MicroCalSingle(object):
    
    def __init__(self, space=0):
        self.set_space(space)
    
    def set_space(self, space):
        self._space = space
        
    def get_pos(self, pos):
        cata = self._space
        cos = CalCosine.cal(cata)
    
        posx = pos[0] + (self._space / 2)
        posy = pos[1] - (cos/2)
        
        return (posx, posy)
        
    def get_size(self, size):
        return (size[0] - self._space, size[1])

    
class PosMatrix3D(object):
    
    def __init__(self, size, pos, off):
        self._size = size
        self._off = off
        self._pos = pos
    
    def __call__(self):
        return self._pos
    
    def __getitem__(self, i):
        return self._pos[i]
    
    
    # ======================================== back mid
    def start_node_back(self, off=(0,0)):
        md = self._start_back()
        
        pos = [self._pos[x] + md[x] - off[x] for x in range(2)]
        return pos
    
    def mid_node_back(self, off=(0,0)):
        md = self._mid_back()
        
        pos = [self._pos[x] + md[x] for x in range(2)]
        return pos
    
    def end_node_back(self, off=(0,0)):
        md = self._end_back()
        
        pos = [self._pos[x] + md[x] + off[x] for x in range(2)]
        return pos
    
    def start_point_back(self, off=(0,0)):
        md = self._start_back()
        om = self._mid_space_back()
        
        pos = [self._pos[x] + md[x] + om[x] - off[x] for x in range(2)]
        return pos
    
    def mid_point_back(self, off=(0,0)):
        md = self._mid_back()
        om = self._mid_space_back()
        
        pos = [self._pos[x] + md[x] + om[x] for x in range(2)]
        return pos
    
    def end_point_back(self, off=(0,0)):
        md = self._end_back()
        om = self._mid_space_back()
        
        pos = [self._pos[x] + md[x] + om[x] + off[x] for x in range(2)]
        return pos
    
    def _start_back(self):
        ss1 = self._size[0] / 2
        cos2 = CalCosine.cal(ss1)
        
        x = 0
        y = self._size[1] - cos2
        
        return (x, y)
    
    def _mid_back(self):
        ss1 = self._size[0] / 4
        cos2 = CalCosine.cal(ss1)
        
        x = self._size[0] / 4
        y = self._size[1] - cos2
        
        return (x, y)
    
    def _end_back(self):
        ss1 = self._size[0] / 4
        cos2 = CalCosine.cal(ss1)
        
        x = self._size[0] / 2
        y = self._size[1]
        
        return (x, y)
    
    def _mid_space_back(self):
        sizex = (self._size[0]) / 4
        cos = CalCosine.cal(sizex)
        
        return (-sizex, cos)
    
    # ======================================== front point
    def start_node_front(self, off=(0,0)):
        md = self._start_front()
        
        pos = [self._pos[x] + md[x] - off[x] for x in range(2)]
        return pos
    
    def mid_node_front(self, off=(0,0)):
        md = self._mid_front()
        
        pos = [self._pos[x] + md[x] for x in range(2)]
        return pos
    
    def end_node_front(self, off=(0,0)):
        md = self._end_front()
        
        pos = [self._pos[x] + md[x] + off[x] for x in range(2)]
        return pos
    
    def start_point_front(self, off=(0,0)):
        md = self._start_front()
        om = self._mid_space_front()
        
        pos = [self._pos[x] + md[x] + om[x] - off[x] for x in range(2)]
        return pos
    
    def end_point_front(self, off=(0,0)):
        md = self._end_front()
        om = self._mid_space_front()
        
        pos = [self._pos[x] + md[x] + om[x] + off[x] for x in range(2)]
        return pos
    
    def mid_point_front(self, off=(0,0)):
        md = self._mid_front()
        om = self._mid_space_front()
        
        pos = [self._pos[x] + md[x] + om[x] for x in range(2)]
        return pos
    
    def _start_front(self):
        ss = self._size[0] / 2
        cos1 = CalCosine.cal(ss)
        
        x = self._size[0] - (self._size[0] / 2)
        y = self._size[1] - (cos1 * 2)
        
        return (x, y)
    
    def _mid_front(self):

        ss = self._size[0] / 2
        cos1 = CalCosine.cal(ss)
        
        x = self._size[0] - (self._size[0] / 4)
        y = self._size[1] - (cos1 * 1.5)
        
        return (x, y)

    def _end_front(self):
        ss = self._size[0] / 2
        cos1 = CalCosine.cal(ss)
        
        x = self._size[0]
        y = self._size[1] - cos1
        
        return (x, y)
    
    def _mid_space_front(self):
        sizex = (self._size[0]) / 4
        cos = CalCosine.cal(sizex)
        
        return (sizex, -cos)
        
        
class Matrix3D(MatrixRotation):

    def rotateXY(self, cad, size=1):
        
        if size > 1:
            nposy = (size - 1) / 2 
            cad = (cad[0], cad[1]+nposy)
        
        res = self.cal_off(cad)
        
        return PosMatrix3D(self._size, res, self._off)

    def rotateNodeXY(self, node):
        cadx, cady, size, _ = node
        return self.rotateXY((cadx, cady), size)
        
    def cal_off(self, pos):
        x = self.cal_offx(*pos)
        y = self.cal_offy(*pos)
        
        return (x, y)
    

class HelperDraw(object):
    
    def __init__(self, size, matrix3d=None):
        self._size = size
        self._matrix3d = matrix3d
        
class HelperDrawBasePolyline(HelperDraw):
        
    def create_polyline_by_pos(self, cad1, cad2):
        pos1 = self._matrix3d.rotateXY(cad1)()
        pos2 = self._matrix3d.rotateXY(cad2)()
        
        return self.append_points(pos1, pos2)
        
    def cal_points(self, pos, pos2):
        sizex = self._size[0]
        h_sizex = sizex / 2
        sizey = self._size[1]
        
        cos = CalCosine.cal(h_sizex)
        
        lst = [
            (pos[0], pos[1] + sizey - cos),
            (pos[0] + h_sizex, pos[1] + sizey - (cos*2)),
            (pos2[0] + sizex, pos2[1] + sizey - cos),
            (pos2[0] + h_sizex, pos2[1] + sizey)
        ]
        
        return lst
    
    def append_points(self, pos1, pos2):
        return self.cal_points(pos1, pos2)

class HelperDrawLabel(HelperDraw):
    
    def label_by_node(self, pos, node, maxl=10):
        name = node.get('name', '-')
        
        name = name[0:maxl]
        ln = len(name)
        
        fsize = self._size[0] * 0.16
        sposy = self._size[1] * 1.18
        
        sposx = (self._size[1] * 0.65) + (ln * (fsize * 0.24))
        
        npos = [-1*pos[x] for x in range(2)]
        npos = (npos[0]-sposx, npos[1]+sposy)
        
        degrees = (-30.404, 41.)
        
        mpos = (pos[0]+15, pos[1])
        transf = (*mpos, *degrees, *npos)
        
        opts = {
            'font_size': fsize,
            'transform': 'translate(%s, %s) skewY(%s) skewX(%s) translate(%s, %s)' % transf}
        
        return name, pos, opts

class ScoreSize(object):

    mindex = [2, 2.5, 3.5, 7, 11, 22, 46, 76, 1000]
    mvalue = ['nano', 'micro', 'small', 'medium', 'large', '2xlarge', '4xlarge', '8xlarge', '8xlarge']
    
    @staticmethod
    def scored(scr):
        valued = None
        
        for key, val in zip(ScoreSize.mindex, ScoreSize.mvalue):
            if scr <= key:
                valued = val
                break             
 
        return valued

class ScoreServer(object):
    
    @staticmethod
    def make_score(cpu, memory):
        return (float(cpu) * 1.5) + float(memory)
    
    @staticmethod
    def val_score(score):
        return ScoreSize.scored(score)
    
    
class DcServers(object):
    
    allowed = ['aws', 'openstack', 'azure']
    
    @staticmethod
    def byServer(server, dft='premise'):
        
        if 'datacenters' in server:
            dc = server.get('datacenters')
            
            if 'provider' in dc:
                provider = dc.get('provider').lower()

                if provider in DcServers.allowed:
                    return provider
                
        return dft

    
class HelperDrawTooltips(HelperDraw):
    
    def __init__(self, size, off):
            
        self._text = []
        self._pos = None
        self._app = None
        self._off = off
        
        self._font_size = 12
        super().__init__(size)
    
    def execute(self, pos, app):
        x = self._size[0] + 5
        y = (self._size[1]) / 3
        
        self._pos = (x, y)
        self._app = app
        
        dc = DcServers.byServer(self._app)
        servers = self._app.get('servers', [0])
        
        props = {k: self._app.get(k, None) for k in ('name', 'provider', 'family', 'environment')}

        self.make_title(props['name'][:20])
        
        self.make_property('DC', dc)
        self.make_property('Env', props['environment'])
        self.make_property('Family', props['family'])
        self.make_property('Provider', props['provider'])
        self.make_property('Servers', len(servers))
     
    def get_background(self):
        sbox = (170, 140)
        
        opts = {
            'class_': "tb-back",
            "rx": "3",
            "ry": "3" 
        }
        
        bg = (self._pos, sbox, opts)
        return bg
    
    def get_button(self):
        sbox = (20, 140)
        npos = (self._pos[0] + 150, self._pos[1])
        
        opts = {
            'fill': "#000",
            "fill-opacity": "0.8",
            "rx": "3",
            "ry": "3" 
        }
        
        btn = (npos, sbox, opts)
        return btn
        
    def get_text(self):
        return (self._text, self._pos)
    
    def make_property(self, key, value, space=(12,3)):
        if key and value: 
            dx = self._pos[0] + space[0]
            dy = self._font_size + space[1]

            label = '> %s: ' % key

            objl = self.make_text(
                label, 
                {'dy': [dy], 'x': [dx], 'font_weight': 'bold'}
            )

            text = str(value).capitalize()
            objv = self.make_text(text)

            self._text.append(objl)
            self._text.append(objv)
    
    def make_title(self, title, space=(12, 14)):
        dx = self._pos[0] + space[0]
        dy = self._font_size + space[1]
        tfont = self._font_size + 2
        
        objt = self.make_text(
            title.capitalize(), 
            {'font_size': tfont, 'dy': [dy], 'x': [dx], 'fill': '#fff', 'font_weight': 'bolder'}
        )
        
        objl = self.make_text(
            '--', 
            {'dy': [self._font_size], 'x': [dx]}
        )
        
        self._text.append(objt)
        self._text.append(objl)
        
    def make_text(self, text, opts={}):
        font_size = opts.get('font_size', self._font_size)
    
        dts = {
            'class_': 'tfont',
            'text': text,
            'font_size': font_size
        }
        
        return {**dts, **opts}
        
    
class HelperDrawApplication(HelperDraw):
    
    def __init__(self, size, servers, microCal=MicroCalSingle):
        
        
        self._microsingle = microCal()
        
        self._apps = []
        self._servers = servers
        self._calGrid = None
        self._pos = None
        self._nsize = None
        self._gsize = None
        
        super().__init__(size)
    
        
    def execute(self, pos, node):
        self._pos = pos
        servers = node.get('servers')
        
        family = node.get('family', 'application').lower()
        
        if servers:
            self.template_with_servers(servers, family)
            
        if not servers:
            self.template_without_servers(family, node)
        
    def template_with_servers(self, servers, family):
        qtd = len(servers)

        self._calGrid = FactoryCalGrid.caller(qtd, self._size)
        mx = self._calGrid.get_size()
        
        self.cal_ajust()

        ordered_servers = self.sorted_servers(servers, family)
        for serv in enumerate(ordered_servers):
            if mx > serv[0]:
                self.single_server(*serv)
             
    def template_without_servers(self, family, node):
        qtd = node.get('qtd', 1)
        catsize = node.get('size', 'medium')
        asset = '%s.%s' % (family, catsize)
        template = DcServers.byServer(node)
        
        self._calGrid = FactoryCalGrid.caller(qtd, self._size)
        self.cal_ajust()
        
        for counter in range(qtd):
            self.draw_applications(counter, asset, template)
    
    def sorted_servers(self, servers, family):
        
        prepared = []
        
        
        for id_server in servers:
            details = self._servers.get(id_server)
            
            if details:
                cpu = details.get('cpu', 1)
                memory = details.get('memory', 1)
                score = ScoreServer.make_score(cpu, memory)

                ss = {k: details.get(k, None) for k in ('hostname', 'ipv4_private', 'ipv4_public', 'datacenters', 'services', 'storage', 'cpu', 'memory', 'environment', 'role', 'os')}
                ss['asset'] = '%s.%s' % (family, ScoreServer.val_score(score))
                ss['score'] = score

                prepared.append(ss)
        
        prepared = sorted(prepared, key=itemgetter('score'), reverse=True)
        return prepared
    
    def single_server(self, counter, server):
        asset = server.get('asset')
        score = server.get('score')

        template = DcServers.byServer(server)
        self.draw_applications(counter, asset, template)
        
    def draw_applications(self, counter, asset, template):
        xpos, ypos = self._calGrid.get_position(counter)

        npos = (self._pos[0] + xpos, self._pos[1] + ypos)
        self.draw_symbol(npos, asset, template)
        
    def draw_symbol(self, pos, asset, template):
        npos = self._microsingle.get_pos(pos)
        nsize = self._microsingle.get_size(self._calGrid.get_nsize())
        
        app = (asset, template, npos, nsize)
        self._apps.append(app)
    
    def cal_ajust(self, base=0.18):
        double = self._calGrid.get_size()
        factor = (double / 100)
        inc = base - factor
        ajust = self._size[0]*inc
        self._microsingle.set_space(ajust)
        
    def get_apps(self):
        return self._apps

    
class PathFactor(object):
    
    def __init__(self):
        self._d = "M"
        
    def __call__(self):
        return self._d.strip()
    
    def add_point(self, x, y):
        self._d += "%s %s " % (x, y)
        

class HelperPointerNodeConnector(object):
    
    def __init__(self, matrix3d, pathFactor=PathFactor):
        self._matrix3d = matrix3d
        self._path = pathFactor()
    
    def get_path(self):
        return self._path()
    
    # front end points
    def node_mid_front(self, pos, size):
        self._factory_matrix(pos, size, 'mid', 'node', 'front')
        
    def point_mid_front(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'mid', 'point', 'front')
     
    def node_start_front(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'start', 'node', 'front', om)
    
    def node_end_front(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'end', 'node', 'front', om)
    
    def point_start_front(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'start', 'point', 'front', om)
        
    def point_end_front(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'end', 'point', 'front', om)
    
    # back end points
    def node_start_back(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'start', 'node', 'back', om)
        
    def node_mid_back(self, pos, size):
        self._factory_matrix(pos, size, 'mid', 'node', 'back')
    
    def point_mid_back(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'mid', 'point', 'back')
        
    def node_end_back(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'end', 'node', 'back', om)
        
    def point_start_back(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'start', 'point', 'back', om)
        
    def point_end_back(self, pos, size, om=(0,0)):
        self._factory_matrix(pos, size, 'end', 'point', 'back', om)
           
        
    def _factory_matrix(self, pos, size, alloc='mid', point='node', view='front', om=(0,0)):
        method = "%s_%s_%s" % (alloc, point, view)
        
        PoxMatrix = self._matrix3d.rotateXY(pos, size)
        npos = getattr(PoxMatrix, method)(om)
        self._path.add_point(*npos)
        
class HelperNodeConnector(object):

    def __init__(self, nd1, nd2):
        self._node = (nd1, nd2)
        self._diffs = None
        self._pos = []
        self._size = None
        
        self.setup()

    def setup(self):
        x1, y1, s1, _ = self._node[0]
        x2, y2, s2, _ = self._node[1]
        
        self._pos.append((x1, y1))
        self._pos.append((x2, y2))
        
        self._size = (s1, s2)
        
        self.diff_nodes()
        
    def diff_nodes(self):
        x1, y1 = self._pos[0]
        x2, y2 = self._pos[1]
        
        dx = x2 - x1
        dy = y2 - y1
        
        self._diffs = (dx-1, dy)
    
    def get_diffs(self):
        return self._diffs
    
    def is_down(self):
        return self._diffs[0] >= 0
    
    def is_double_down(self):
        return self._diffs[0] >= 1
    
    def is_up(self):
        return self._diffs[0] < 0
    
    def do_first_node(self, x=0, y=0):
        return self._pos[0]

    def do_second_node(self, x=0, y=0):
        return self._pos[1]
    
    def get_first_size(self):
        return self._size[0]
     
    def get_second_size(self):
        return self._size[1]
    
class HelperDrawConnector(HelperDraw):
    def __init__(self, size, off, matrix3d=None, HPointer=HelperPointerNodeConnector):
        super().__init__(size, matrix3d)
        
        self._off = off
        
        self._nodes = []
        self._hconnector = None
        self._hpointer = HPointer(matrix3d)

    
    def connect(self, node1, node2, HNodeConnector=HelperNodeConnector):
        
        self._hconnector = HNodeConnector(node1, node2)
        
        fpos = self._hconnector.do_first_node()
        fsize = self._hconnector.get_first_size()
        spos = self._hconnector.do_second_node()
        ssize = self._hconnector.get_second_size()
        
        diffs = self._hconnector.get_diffs()
        
        self._hpointer.node_mid_front(fpos, fsize)
        
        
        if self._hconnector.is_down():
            self.first_step(fpos, spos, fsize, ssize, diffs)
                   
        if self._hconnector.is_double_down():
            self.second_step(fpos, fsize, diffs) 
                
        if self._hconnector.is_up():
            self.revert_step(fpos, fsize, diffs)
            
        
        self._hpointer.point_mid_back(spos, ssize)
        self._hpointer.node_mid_back(spos, ssize)
        
        
        return self._hpointer.get_path()
    
    def first_step(self, fpos, spos, fsize, ssize, diffs):
        if fsize > 1 or diffs[0] == 0:
            self.ajust_corner(fpos, spos, fsize, ssize)
        else:    
            self._hpointer.point_mid_front(fpos, fsize)
    
    def second_step(self, fpos, fsize, diffs):
        cx = fpos[0]+1
        cy = fpos[1]

        if (cx in self._matrix3d._grid) and (cy in self._matrix3d._grid[cx]):
            check = self._matrix3d._grid[cx][cy][0]

            if check == '-':
                self._hpointer.point_mid_front(fpos, 1)
                self._hpointer.node_mid_front((fpos[0]+diffs[0], fpos[1]), 1)

            if check != '-':
                m = self.sizes(10, 40)

                self._hpointer.point_start_front(fpos, 1, m)
                self._hpointer.node_start_front((fpos[0]+diffs[0], fpos[1]), fsize, m)
    
    def revert_step(self, fpos, fsize, diffs):
        m = self.sizes(12, 40)

        if diffs[1] <= 0:
            self._hpointer.point_start_front(fpos, 1, m)
        else:
            self._hpointer.point_end_front(fpos, 1, m)

        self._hpointer.point_start_front((fpos[0], fpos[1] + diffs[1]), 1, m)
        self._hpointer.point_start_front((fpos[0] + diffs[0], fpos[1] + diffs[1]), 1, m)
            
    def ajust_corner(self, fpos, spos, fsize, ssize):
        _, y1 = self._matrix3d.rotateXY(fpos, fsize)()
        _, y2 = self._matrix3d.rotateXY(spos, ssize)()

        dy = y2 - y1

        cal = (self._size[1] * self._off[1]) / 5
        cal2 = cal / 2

        if dy >= -cal2:
            self._hpointer.point_end_front(fpos, fsize)
            return

        if dy <= -cal:
            self._hpointer.point_start_front(fpos, fsize)
            return
    
    
    def sizes(self, dx=1, dy=1):
        sx = self._size[0] * self._off[0]
        sy = self._size[1] * self._off[1]
        
        return (sx / dx, sy / dy)
    

        
class DrawTemplateSVG(object):
    def __init__(self, tmax, servers, grid, darea=DrawArea, symbols=SymbolAssets, m3d=Matrix3D):
        self._off = (2.2, 2.5)
        size = 100
        self._size = (size, size*1.9)

        self._matrix3d = m3d(tmax[0], self._size, self._off, grid)
        
        self._area = darea(self._off, self._size, tmax[0], tmax[1], grid).area()

        self.dwg = Drawing('test.svg', size=self._area)
        self._symbols = symbols(self.dwg)
        self._servers = servers
        
        self.setup()
    
    def setup(self):
        self.setup_background()
        self.setup_brightness()
        self.setup_marker()
    
    def setup_background(self):
        symbol = self._symbols.square((0,0), size=self._area, opts={'fill': "#eaeaea"})
        self.add(symbol)
    
    def setup_brightness(self):
        self._symbols.brightness()
        self._symbols.brightness("darker", 0.4)
    
    def setup_marker(self):       
        s = self._size[0] / 8
        
        opts = {
            'insert': (152, 3), 
            'size': (s, s)
        }
        
        self._symbols.asset_marker('markers.arrow', opts)
        
    def boundary_box(self, pos):
        symbol = self._symbols.asset('boundaries_box.front', 'default', (pos[0], pos[1]), self._size)
        self.add(symbol)
        
    def draw_label(self, pos, node):
        
        text = HelperDrawLabel(self._size, self._matrix3d)\
                        .label_by_node(pos, node)
        
        symbol = self._symbols.text(*text)
        self.add(symbol)
        
        
    def draw_app(self, item):
        cad1 = [item[x] for x in range(2)]
        node = item[3]
        
        pos = self._matrix3d.rotateNodeXY(item)()
    
        self.draw_grid_size(cad1, item[2])
        self.grid_box(pos)
        self.draw_execute(pos, node)
        self.boundary_box(pos)
        self.draw_label(pos, node)
        self.draw_tooltips(pos, node)
    
    def draw_execute(self, pos, node):
        hDrawApp = HelperDrawApplication(self._size, self._servers)
        hDrawApp.execute(pos, node)
        
        pSymb = hDrawApp.get_apps()
        
        for symb in pSymb:
            symbol = self._symbols.asset(*symb)
            self.add(symbol)
    
    def draw_tooltips(self, pos, node):
        _id = node.get('_id')
        g = self._symbols.create_group(_id)
        
        hDrawTooltips = HelperDrawTooltips(self._size, self._off)
        hDrawTooltips.execute(pos, node)
        
        background = hDrawTooltips.get_background()
        symbol = self._symbols.square(*background)
        g.add(symbol)
        
        background = hDrawTooltips.get_button()
        symbol = self._symbols.square(*background)
        g.add(symbol)
        
        ltxt = hDrawTooltips.get_text()
        symbol = self._symbols.multiline(*ltxt)
        g.add(symbol)
        
    def grid_box(self, pos, opts={'fill-opacity': '0.4'}):
        symbol = self._symbols.asset('grid.base', 'default', pos, self._size, opts)
        self.add(symbol)
        
    
    def draw_grid_size(self, cad1, size):

        cad2 = (cad1[0], cad1[1]+size-1)
        
        points = HelperDrawBasePolyline(self._size, self._matrix3d)\
                    .create_polyline_by_pos(cad1, cad2)
        
        symbol = self._symbols.polyline(points, {'fill': '#ccc', 'fill-opacity': 0.2})
        self.add(symbol)
    
        
    def add(self, symbol):
        self.dwg.add(symbol)
        
    def save(self):
        self.dwg.save()
        return self.dwg.get_xml()
        
    def draw_connect(self, node1, node2, details={}):
        d = HelperDrawConnector(self._size, self._off, self._matrix3d).connect(node1, node2)
        
        symbol = self._symbols.conn(d)
        self.add(symbol)
        
    
class DrawLayout(object):
    def __init__(self, grid, index, servers={}, draw=DrawTemplateSVG):
        
        self._grid = grid
        self._index = index
        
        self._max_x = max(self._grid, key=int)
        self._max_y = max(self._grid[0], key=int)
        self.setup_drawer(draw, servers)
        

    def setup_drawer(self, draw, servers):
        tmax = (self._max_x, self._max_y)
        self.drawer = draw(tmax, servers, self._grid)
        
    def draw_nodes(self):
        for _, node in self._index.items():
            self.drawer.draw_app(node)
                
        return self
        
    def draw_connections(self, edges):
        for edge in edges:
            edg = [self._index[edge[x]] for x in range(2)]
            
            self.drawer.draw_connect(*edg, edge[2])
            
        return self
    
    def save(self):
        return self.drawer.save()

In [11]:
from IPython.display import SVG, display
from functools import reduce
from app.repository.externalMaestroData import ExternalMaestroData

print("-------------------------")
Orchestration = GridOrchestrator(network.graph)
Orchestration.create(entries)

servers_id = reduce(append_servers, Orchestration.get_grid().get_index().values(), [])
servers_id = list(set(servers_id)) #remove duplicate
query = {"_id": servers_id}
ExternalRequest = ExternalMaestroData(owner_id=rolesa, graph_id="asda")
result = ExternalRequest.get_request(path="servers", query=query)

servers = transform_dict(result)

Layout = DrawLayout(Orchestration.get_grid().get_grid(), Orchestration.get_grid().get_index(), servers)

Layout.draw_connections(network.graph.edges(data='endpoint'))
Layout.draw_nodes()

Layout.save()
   
display(SVG(filename='test.svg'))

-------------------------
